In [ ]:
from nbdev import *
%nbdev_default_export control.functions

Cells will be exported to pct.control.functions,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
#hide
#%load_ext autoreload
#%autoreload 2

In [ ]:
import numpy as np

ModuleNotFoundError: No module named 'numpy'

# Functions

Functions that form the elements of a perceptual control node (system).

In [ ]:
%nbdev_export
from abc import ABC, abstractmethod

In [ ]:
%nbdev_export
class BaseFunction(ABC):
    "Base class of a PCT function."
    def __init__(self):
        self.output = np.zeros(1)
        
    @abstractmethod
    def __call__(self):
        pass
    
    def set_output(self, output):
        self.output= output
    
    def get_output(self):
        return self.output

In [ ]:
%nbdev_export
class Proportional(BaseFunction):
    "Proportional function."
    def __init__(self, gain):
        super().__init__()
        self.gain = gain
    
    def __call__(self, input):
        self.output = input * self.gain
        return self.output

In [ ]:
%nbdev_export
class Integration(BaseFunction):
    "Integration function."
    def __init__(self, gain, slow):
        super().__init__()
        self.gain = gain
        self.slow = slow
    
    def __call__(self, input):
        self.output = self.output +  ((input * self.gain) - self.output)/self.slow
        return self.output
 

In [ ]:
integrator = Integration(3, 10)
output = integrator(5)
print(output)

NameError: name 'np' is not defined

In [ ]:
#hide
integrator.set_output(0)
assert integrator(5) == 1.5

In [ ]:
#hide
from nbdev import *
notebook2script()